# This notebook saves CAFE v1 atmospheric daily climatologies in a single dataset

#### IMPORTANT: If adapting this code to save other climatologies, be careful to only include full years, as pyLatte will compute monthly climatologies from the saved daily climatologies

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from pylatte import utils
from ipywidgets import FloatProgress

#### Initialise

In [2]:
# Location of forecast data -----
fcst_folder = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/'
fcst_filename = 'atmos_daily*'

fields = pd.DataFrame( \
        {'name_CAFE': ['hght', 'sphum', 'temp', 'ucomp', 'vcomp'],
         'name_std' : ['gh',   'sphum', 'temp', 'u',     'v']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields = pd.DataFrame( \
        {'name_CAFE': ['lwflx', 'shflx', 'tau_x', 'tau_y', 't_ref', 'q_ref', 'u_ref', 'v_ref', 't_ref_min',
                       't_ref_max', 't_surf', 'ps',  'slp', 'h500', 'precip', 'lwdn_sfc', 'lwup_sfc', 'olr',  
                       'swdn_sfc', 'swup_sfc', 'swup_toa',   'high_cld_amt', 'low_cld_amt', 'mid_cld_amt', 'tot_cld_amt'],
         'name_std' : ['lwf',   'shf',   'tau_x', 'tau_y', 't_ref', 'q_ref', 'u_ref', 'v_ref', 't_ref_min',
                       't_ref_max', 't_s',    'p_s', 'slp', 'h500', 'precip', 'lwf_dn_s', 'lwf_up_s', 'olwr', 
                       'swf_dn_s', 'swf_up_s', 'swf_up_toa', 'high_cld_amt', 'low_cld_amt', 'mid_cld_amt', 'tot_cld_amt']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields


,name_CAFE,name_std
0,precip,precip


In [3]:
# Partial lists -----

fields = pd.DataFrame( \
        {'name_CAFE': ['hght', 'sphum', 'temp', 'ucomp', 'vcomp'],
         'name_std' : ['gh',   'sphum', 'temp', 'u',     'v']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields = pd.DataFrame( \
        {'name_CAFE': ['t_ref_min', 't_ref_max', 't_surf', 'ps',  'precip', 'lwdn_sfc', 'lwup_sfc', 'olr',  
                       'swdn_sfc', 'swup_toa',   'high_cld_amt', 'low_cld_amt', 'mid_cld_amt', 'tot_cld_amt'],
         'name_std' : ['t_ref_min', 't_ref_max', 't_s',    'p_s', 'precip', 'lwf_dn_s', 'lwf_up_s', 'olwr', 
                       'swf_dn_s', 'swf_up_toa', 'high_cld_amt', 'low_cld_amt', 'mid_cld_amt', 'tot_cld_amt']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields = pd.DataFrame( \
        {'name_CAFE': ['precip'],
         'name_std' : ['precip']}
                     )
name_dict = fields.set_index('name_CAFE').to_dict()['name_std']

fields

,name_CAFE,name_std
0,precip,precip


In [4]:
# Initial dates to include (takes approximately 1 min 30 sec per date) -----
init_dates = pd.date_range('2002-2','2016-5' , freq='1MS')

# Ensembles to include -----
ensembles = range(1,12)

#### Load one 366 day long year to provide time array

In [5]:
path = fcst_folder + '/yr2016/mn1/OUTPUT.1/' + fcst_filename + '.nc'
dataset = xr.open_mfdataset(path, autoclose=True)
time_use = dataset.time[:366]

#### Save each year and variable separately due to memory considerations

In [6]:
years = range(2002,2017)
months = range(1,13)
ensembles = range(1,12)

for year in years:
    print(year)
    print('----------')
    for idx, variable in enumerate(fields['name_CAFE']):
        print(variable)
        
        savename = 'cafe.fcst.v1.atmos.' + fields['name_std'][idx] + '.' + str(year) + '.clim.nc'
        try:
            temp = xr.open_mfdataset('/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/' + savename, autoclose=True)
            print('    Already exists')
        except:
            
            fcst_list = []
            for month in months:

                ens_list = []
                ens = []
                empty = True
                for ie, ensemble in enumerate(ensembles):

                    path = fcst_folder + '/yr' + str(year) + '/mn' + str(month) + \
                           '/OUTPUT.' + str(ensemble) + '/' + fcst_filename + '.nc'

                    # Try to stack ensembles into a list -----
                    try:
                        dataset = xr.open_mfdataset(path, autoclose=True)[variable]
                        if 'latb' in dataset.dims:
                            dataset = dataset.rename({'latb':'lat_2','lonb':'lon_2'})
                        ens_list.append(dataset.rename(fields['name_std'][idx]))
                        ens.append(ie+1)
                        empty = False
                    except OSError:
                        # File does not exist -----
                        pass

                # Concatenate ensembles -----
                if empty == False:
                    ens_object = xr.concat(ens_list, dim='ensemble')
                    ens_object['ensemble'] = ens

                    # Stack concatenated ensembles into a list for each month in a year -----                       
                    fcst_list.append(ens_object)

            # Concatenate all months within year -----
            ds = xr.concat(fcst_list, dim='time')

            # Rechunk for chunksizes of at least 1,000,000 elements -----
            ds = utils.prune(ds.chunk(chunks={'ensemble' : len(ds.ensemble), 
                                              'time' : len(ds.time)}).squeeze())

            # Make month_day array of month-day -----
            m = np.array([str(i).zfill(2) + '-' for i in ds.time.dt.month.values])
            d = np.array([str(i).zfill(2)  for i in ds.time.dt.day.values])
            md = np.core.defchararray.add(m, d)

            # Replace time array with month_day array and groupby -----
            ds['time'] = md
            ds_clim = ds.groupby('time').mean(dim=['time','ensemble'],keep_attrs=True)

            # Fill time with presaved time -----
            ds_clim['time'] = time_use
            ds_clim.time.attrs['long_name'] = 'time'
            ds_clim.time.attrs['cartesian_axis'] = 'T'
            ds_clim.time.attrs['calendar_type'] = 'JULIAN'
            ds_clim.time.attrs['bounds'] = 'time_bounds'

            # Save and delete -----
            with utils.timer():
                ds_clim.to_netcdf(path='/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/' + savename,
                                  mode = 'w',
                                  encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                                                      'units':'days since 0001-01-01 00:00:00'}}) 

            del ds, ds_clim

2002
----------
precip
   Elapsed: 1521.167345046997 sec
2003
----------
precip
   Elapsed: 4173.270169973373 sec
2004
----------
precip
   Elapsed: 3911.1967186927795 sec
2005
----------
precip
   Elapsed: 3932.8436381816864 sec
2006
----------
precip
   Elapsed: 3420.890709400177 sec
2007
----------
precip
   Elapsed: 3781.8179755210876 sec
2008
----------
precip
   Elapsed: 3895.825021982193 sec
2009
----------
precip
   Elapsed: 3798.0376267433167 sec
2010
----------
precip
   Elapsed: 2554.4368908405304 sec
2011
----------
precip
   Elapsed: 1878.5482606887817 sec
2012
----------
precip
   Elapsed: 2082.0910065174103 sec
2013
----------
precip
   Elapsed: 1938.466807126999 sec
2014
----------
precip
   Elapsed: 1221.2286267280579 sec
2015
----------
precip
   Elapsed: 1203.7032833099365 sec
2016
----------
precip
   Elapsed: 560.9949703216553 sec
